# Setup

In [14]:
from PIL import ImageGrab
import win32gui, win32com.client
import cv2
import numpy as np
import pytesseract
import time

In [2]:
tessdata_dir_config = '--tessdata-dir "C:/msys64/mingw64/share/tessdata"'

In [3]:
#Get hwnd for dota 2 client window
toplist, winlist = [], []
def enum_cb(hwnd, results):
    winlist.append((hwnd, win32gui.GetWindowText(hwnd)))
win32gui.EnumWindows(enum_cb, toplist)

dota_window = [(hwnd, title) for hwnd, title in winlist if title == "Dota 2"]
dota_window = dota_window[0]
hwnd = dota_window[0]

vscode = [(hwnd, title) for hwnd, title in winlist if 'visual studio code' in title.lower()]
vscode_hwnd = vscode[0][0]

# Screenshot

In [4]:
def dota_screenshot():
    #Bring dota 2 window to the front, take screenshot 
    shell = win32com.client.Dispatch("WScript.Shell")
    shell.SendKeys('%')
    win32gui.SetForegroundWindow(hwnd)

    bbox = win32gui.GetWindowRect(hwnd)
    img = ImageGrab.grab(bbox)

    #img.show()
    #img.save("./imgs/test_diff.png")
    #win32gui.SetForegroundWindow(vscode_hwnd)

    return img


In [ ]:
CARD_BBOX = (808, 298, 1111, 722)
cropped = img.crop(CARD_BBOX)
cropped.save("./imgs/card.png")

In [ ]:
image = cv2.imread("./imgs/test.png")
template = cv2.imread("./imgs/activated_button.png")

result = cv2.matchTemplate(image,template,cv2.TM_CCOEFF_NORMED)
print(np.unravel_index(result.argmax(),result.shape))

In [48]:
img = dota_screenshot()
time.sleep(.4)
win32gui.SetForegroundWindow(vscode_hwnd)
img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)

In [49]:
y, h = 294, 433
x, w = 280, 1360
#Load image
original = img.copy()

#Preprocess - crop, grayscale, threshold
#img = img[y:y+h, x:x+w]
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(gray, 35, 255, cv2.THRESH_BINARY)

In [50]:
contours, hierarchy = cv2.findContours(image=thresh, mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_SIMPLE)
cards_bbox = []
image_copy = img.copy()
#cv2.drawContours(image=image_copy, contours=contours, hierarchy = hierarchy, contourIdx=-1, color=(0, 255, 0), thickness=2, lineType=cv2.LINE_AA, maxLevel=1)
for i in range(len(contours)):
    c = contours[i]
    if hierarchy[0][i][-1] < 0:
        cards_bbox.append(cv2.boundingRect(c))
        x,y,w,h = cv2.boundingRect(c)
        cv2.rectangle(image_copy, (x, y), (x + w, y + h), (36,255,12), 2)

cv2.imshow('None approximation', image_copy)
cv2.waitKey(0)
cv2.imwrite('contours_none_image1.jpg', image_copy)
cv2.destroyAllWindows()


In [42]:
cards = []
for bbox in cards_bbox:
    x,y,w,h = bbox
    print(h*w)
    card = img[y:y+h, x:x+w].copy()
    cards.append(card)
print(1920*1080)

80640
80640
126000
2073600


In [44]:
cv2.imshow('a', cards[-1])
cv2.waitKey(0)
cv2.destroyAllWindows()

In [45]:
for card in [cards[-1]]:
    img_rgb = cv2.cvtColor(card, cv2.COLOR_BGR2RGB)
    out = pytesseract.image_to_string(img_rgb, config = tessdata_dir_config)
    print(out)

 

4

old eLeVeN Vici Gaming

Core b

GOLD CARD

Kills +20%
Roshan Kills 45%
‘Teamfight 425%
‘Obs Wards Planted — +10%
RunesGrabbed 15H,



# Read text

In [102]:
img_cv = cv2.imread("./imgs/card.png")

# By default OpenCV stores images in BGR format and since pytesseract assumes RGB format,
# we need to convert from BGR to RGB format/mode:
img_rgb = cv2.cvtColor(img_cv, cv2.COLOR_BGR2RGB)
out = pytesseract.image_to_string(img_rgb, config = tessdata_dir_config)

In [106]:
out.split('\n')

['Support',
 '',
 'SILVER CARD',
 'Creep Score',
 'GPM',
 'Teamfight',
 '',
 ' ',
 '',
 'Evil Geniuses',
 '',
 '#10%',
 '+#10%',
 '15%',
 '\x0c']

In [ ]:
from PIL import ImageChops, Image

im1 = Image.open("./imgs/test_diff.png")
im2 = Image.open("./imgs/test.png")

diff = ImageChops.difference(im2, im1)
diff

In [ ]:
image = cv2.imread("./imgs/test.png")
template = cv2.imread("./imgs/deactivated_button.png")

result = cv2.matchTemplate(image,template,cv2.TM_CCOEFF_NORMED)
print(np.unravel_index(result.argmax(),result.shape))